# Criando uma Blockchain simples

## Projeto de "Tópicos especiais em sistemas de computação"

Seguindo o tutorial disponível em: https://levelup.gitconnected.com/creating-a-blockchain-from-scratch-9a7b123e1f3e

Para entender o comportamento geral de uma blockchain simples, consideremos uma blockchaincomo uma base de dados. Por exemplo, podemos pensar que em um:
- Formulário: queremos armazenar dados que fornecemos e buscamos quando necessário;
- Blog: tal como o formulário, queremos fornecer, armazenar e buscar dados quando desejarmos;
- Loja: como um meio de transações, queremos compartilhar dados (o produto em si, sua disponibilidade e preço) e queremos recuperar informações sobre pagamento e endereço do comprador;

Com isso, consideremos um bloco (componentes de uma blockchain) como uma caixa de conteúdos que gerenciamos diretamente: os dados (ID do produto, disponibilidade e preço, se cosideramos uma loja) e uma data, por exemplo. Por último, ela deve conter um rótulo criptografico que será anexado para manter sua segurança.

In [1]:
from datetime import datetime
import hashlib

In [2]:
# Initialize a block to place within the blockchain

# The created block must have:
#     data (to be stored within the decentralized chain network)
#     previous_block_hash (to be attached to to grant security)

class Block():
    def __init__(self, data, previous_block_hash):
        
        # reserve the time stamp when the block is required to be created
        self.timestamp = datetime.utcnow()
        
        # Store the data put inside the block
        self.data = data
        
        # Store the encoded cryptographic label from previous block
        self.prev_block_hash = previous_block_hash
        
        # Calculate the cryptography for the current block
        self.hash = hashlib.sha256(self.to_string().encode()).hexdigest()
        
        # Recalculate the cryptography to Proof-of-Work (PoW)
        self.calculate_valid_hash()
        
    def to_string(self):
        
        # Return the block specificity:string containing data + timestamp + prev. encryptation
        return "{}\t{}\t{}".format(self.data, self.timestamp, self.prev_block_hash)
    
    def is_hash_valid(self, _hash):
        
        # Return if the recalculated encryptation is valid (PoW)
        return (_hash.startswith('0'*3))
    
    def calculate_valid_hash(self):
        
        # Initialize a empty string as a encrypted label
        _hash = ''
        
        # Initialize a bit-like disturbance over the created cryptography
        nonce = 0
        
        while (not self.is_hash_valid(_hash)):
            
            # Store a temporary variable with the block specific string 
            # (data + timestamp + prev. encryptation) and add the bit-like disturbance
            temp = self.to_string() + str(nonce)
            #print(temp)
            
            # Recalculate the cryptography considering the bit-like disturbance
            _hash = hashlib.sha256(temp.encode()).hexdigest()
            
            # Increment bit-like the disturbance
            nonce += 1
            
        # Once done, store the current block's cryptographic label 
        self.hash = _hash
        
        # Store the nonce as static to be easily accessed
        self.nonce = nonce

In [3]:
# Create a blockchain. The network enlacing the uploaded blocks
# For each block, the blockchain stores:
#     data, timestamp, cryptographic label

class Blockchain():
    def __init__(self):
        
        # Initialize the blockchain as a array
        self.blocks = []
        
        # Place the first block inside the blockchain
        self.set_genesis_block()
        
    def set_genesis_block(self):
        
        # Generate the data from the first block
        data = 'Genesis'
        
        # Generate the first cryptogtaphic label (trivial)
        prev_hash = '0'*64
        
        # Generate a block container with: data + prev. encryptation
        genesis_block = Block(data, prev_hash)
        
        # Add the block to the blockchain
        self.blocks.append(genesis_block)
        
    def get_last_hash(self):
        
        # Get the last block placed in the blockchain
        last_block = self.blocks[-1]
        
        # Get the last encryption label
        last_hash = last_block.hash
        
        # Return the last encrpytion label
        return (last_hash)
    
    def add_new_block(self, data):
        
        # Get the last encryption label (from the last block added to the blockchain)
        prev_hash = self.get_last_hash()
        
        # Generate the block with the provided date
        new_block = Block(data, prev_hash)
        
        # Add the new block to the blockchain
        self.blocks.append(new_block)
        

## Entendendo o funcionamento da Blockchain

In [4]:
blockchain = Blockchain()

Quando inicializamos nossa blockchain, criamos uma lista contendo um bloco, nosso bloco "Genesis". Ele contém o seguinte:

In [5]:
blocks = blockchain.blocks
print('Blockchain:\n\n', blocks[0].to_string())

Blockchain:

 Genesis	2021-05-06 21:52:08.475792	0000000000000000000000000000000000000000000000000000000000000000


Esses dados representam o "Dado" contido no bloco, a "data" de criação do bloco e seu "rótulo criptográfico". Depois disso, podemos adicionar outros blocos:

In [6]:
blockchain.add_new_block('First block')
blockchain.add_new_block('Second block')

blocks = blockchain.blocks
print('Blockchain:\n\n', blocks[0].to_string(), '\n', 
                         blocks[1].to_string(), '\n', 
                         blocks[2].to_string())

Blockchain:

 Genesis	2021-05-06 21:52:08.475792	0000000000000000000000000000000000000000000000000000000000000000 
 First block	2021-05-06 21:54:38.789053	000577464050f5ffef5ad044a27ae84a8733012130e1da3dd7c8533650f5f19a 
 Second block	2021-05-06 21:54:38.810896	0006c82cba25702389eea429f6bab3012c87767eb2e3fec932c54535c694028a


Em um blockchain com múltiplos usuários, adições simultâneas poder ser feitas e blocos multi conectados podem existir. Entretanto, aqui estamos adicionando blocos sequencialmente. Logo:

- bloco 1 é criado e conectado ao bloco 0;
- bloco 2 é criado e conectado ao bloco 1;
- bloco 3 é criado e conectado ao bloco 2;
- e assim por diante.

Para executar a conexão, o a encriptação do primeiro bloco deve depender do rótulo criptográfico do 'Genesis'. Aligação é feita da seguinte forma:

1. definir uma função criptográfica que será usada no Blockchain. Aqui usamos sha256;
2. a função deve converter alguma entrada de texto em um texto criptografado. Aqui codificamos uma sequência com 64 espaços para combinações de {0-9, a-z};
3. o texto de entrada deve conter pelo menos o rótulo criptográfico do bloco anterior. Aqui combinamos 'dado' + 'data' + 'rótulo criptográfico' e usamos esse novo texto para gerar um rótulo codificado para o bloco adicionado;
4. encoragar a integridade da blockchain utilizando um protocolo de consenso. Aqui utilizamos algoritmo de Proof-of-Work (PoW) ou Prova-de-Trabalho (PdT). Isso faz da criação de blocos uma tarefa custosa, porém recompensada financeiramente.

On a blockchain with multiple users, simultaneous additions could be make and a multi-connected block would exist. However, here we are adding the blocks sequentially. Therefore 

- block 1 is created connected to block 0; 
- block 2 is created connected to block 1; 
- block 3 is created connected to block 2; 
- and so on.

O protocolo de consenso Proof-of-Work (PoW) usado aqui adiciona um número incremental ao fim da entrada de texto e apenas aceita um novo bloco caso o rótulo criptográfico gerado comece com '000'.

Por exemplo, sabemos que o bloco 'Genesis' Apresenta as seguintes marcações:

In [7]:
print(blocks[0].to_string())

Genesis	2021-05-06 21:52:08.475792	0000000000000000000000000000000000000000000000000000000000000000


Assim, esperamos que ao usar estes três elementos somente seja suficiente para gerar o rótulo criptográfico do primeiro bloco 'First block'. Vamos chamar esse rótulo de 'raw encoded label', ou 'rótulo codificado cru'

In [8]:
encoded_label = hashlib.sha256(blocks[0].to_string().encode()).hexdigest()
print('Raw encoded label for the First block:\n\n', encoded_label)

Raw encoded label for the First block:

 58deb91497e4b485a6acd0279ef0524a66a67839612be4367fc324eee227c896


Veja que o rótulo real é diferente:

In [9]:
print(blocks[1].to_string())

First block	2021-05-06 21:54:38.789053	000577464050f5ffef5ad044a27ae84a8733012130e1da3dd7c8533650f5f19a


Para gerar considerar o protocolo de consenso por Proof-of-Work (PoW), na verdade precisamos de um mensageiro (ou núncio, vindo da palavra 'nonce') que carregue um número incremental que será adicionado várias vezes ao final do texto de entrada para validar o novo bloco

Podemos pegar o núncio do bloco 'Genesis':

In [10]:
print("Genesis block:\n\n",'Nonce:', blocks[0].nonce)

Genesis block:

 Nonce: 53


Sabemos então que no texto original incrementamos o valor 52 (o primeiro incremento é 0). Logo temos que

In [13]:
nonced_string = blocks[0].to_string() + str(blocks[0].nonce - 1)
print('Nonce string:\n\n', nonced_string)

Nonce string:

 Genesis	2021-05-06 21:52:08.475792	000000000000000000000000000000000000000000000000000000000000000052


é o rótulo criptográfico real que utilizamos para criar o próximo bloco 'First block'

In [15]:
nonced_string = blocks[0].to_string() + str(blocks[0].nonce - 1) # The -1 is considered because the validation occurs
                                                                 # after the unity increment
print('True string to encoded for the first block added:\n\n', nonced_string)

encoded_label = hashlib.sha256(nonced_string.encode()).hexdigest()
print('True encoded label for the First block:\n\n', encoded_label)

True string to encoded for the first block added:

 Genesis	2021-05-06 21:52:08.475792	000000000000000000000000000000000000000000000000000000000000000052
True encoded label for the First block:

 000577464050f5ffef5ad044a27ae84a8733012130e1da3dd7c8533650f5f19a


E vemos que agora o rótulo real gerado a partir do rótulo do bloco 'Genesis' confere com o rótulo do 'First block'

In [16]:
print(blocks[1].to_string())

First block	2021-05-06 21:54:38.789053	000577464050f5ffef5ad044a27ae84a8733012130e1da3dd7c8533650f5f19a


Com o protocolo de consenso Proof-of-Work (PoW), novos blocos são custosos para serem adicionados, mas essa ação recompensa mineradores (aqueles que investem dinheiro para minerar blocos e ganham dinheiro para manter a estrutura do blockchain. Por exemplo em bitcoins) e a conexão entre blocos é reforçada e os dados armazenados nos blocos são melhor validados por auditorias.[1]

[1] Para mais informações sobre PoW: https://www.frontiersin.org/articles/10.3389/fbloc.2020.00002/full